In [1]:
from selenium import webdriver # 버튼 클릭 관련 기능 셀레니움이 크롬 드라이버를 운전해서 크롬을 움직일것임
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from datetime import datetime
import pandas as pd
import time

In [46]:
# 윈도우에서 실행
driver = webdriver.Chrome()
driver.get("https://www.opinet.co.kr/user/main/mainView.do")
time.sleep(2)

# 요소 찾기
menuSan_element = driver.find_element(By.CSS_SELECTOR, "#header > div > ul > li:nth-child(1) > a")
menuRegion_option = driver.find_element(By.CSS_SELECTOR, "#header > div > ul > li:nth-child(1) > ul > li:nth-child(1) > a")

# 메뉴 마우스 호버 -> 지역 호버 -> 클릭
actions = ActionChains(driver)
actions.move_to_element(menuSan_element).perform()  # 유가정보에 마우스 올림
time.sleep(1)
actions.move_to_element(menuRegion_option).click().perform()  # 지역별로 이동해서 클릭
time.sleep(2)

# 구 목록
gu_list = [
    "강서구", "금정구", "기장군", "남구", "동구", "동래구", 
    "부산진구", "북구", "사상구", "사하구", "서구", "수영구", 
    "연제구", "영도구", "중구", "해운대구"
]

# 모든 구에 대해 반복
for gu in gu_list:
    gu_select = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "SIGUNGU_NM0"))
    )
    
    gu_select = Select(gu_select)
    gu_select.select_by_visible_text(gu)  # 구 이름으로 선택
    time.sleep(2)  # 선택 반영 대기

    # 엑셀 저장 버튼 클릭
    excel_btn = driver.find_element(By.CSS_SELECTOR, "#templ_list0 > div:nth-child(7) > div > a")
    excel_btn.click()
    time.sleep(5)  # 엑셀 파일 다운로드를 위한 대기시간

driver.quit()

In [21]:
# 도커에서 실행
from io import BytesIO
import pandas as pd
from hdfs import InsecureClient

hdfs = InsecureClient("http://namenode:9870", user="hadoop")
paths = [
    "강서구.xls",
    "금정구.xls",
    "기장군.xls",
    "남구.xls",
    "동구.xls",
    "동래구.xls",
    "부산진구.xls",
    "북구.xls",
    "사상구.xls",
    "사하구.xls",
    "서구.xls",
    "수영구.xls",
    "연제구.xls",
    "영도구.xls",
    "중구.xls",
    "해운대구.xls",
]
col_names = ["지역", "상호", "주소", "상표", "전화번호", "셀프여부", "고급휘발유", "휘발유", "경유", "실내등유"]

datas = []

for path in paths:
    with hdfs.read(path) as reader:
        bytes_data = reader.read()
        excel_buffer = BytesIO(bytes_data)
        df = pd.read_excel(excel_buffer, header=None, skiprows=2, engine="xlrd")
        datas.append(df)

result_df = pd.concat(datas, ignore_index=True)

result_df.columns = col_names

# 통합 원본
result_df.head(10)

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
1,부산광역시,신창에너지㈜직영 송정셀프주유소,부산 강서구 낙동남로 127 (송정동),S-OIL,051-972-5104,Y,-,1549,1409,-
2,부산광역시,해안주유소,부산 강서구 낙동남로 432,SK에너지,070-8894-3569,Y,-,1549,1399,-
3,부산광역시,은마석유 경기장주유소,부산 강서구 낙동북로 364,GS칼텍스,051-971-7956,Y,1898,1557,1417,1300
4,부산광역시,서강주유소,부산 강서구 낙동북로 42 (강동동),SK에너지,051-971-8484,Y,-,1557,1417,-
5,부산광역시,동방석유㈜직영 대저주유소,부산 강서구 낙동북로 209 (대저1동),SK에너지,051-971-2135,Y,-,1557,1417,-
6,부산광역시,우리2주유소,부산 강서구 낙동북로 142,알뜰주유소,051-971-5585,Y,-,1557,1417,-
7,부산광역시,CJ대한통운㈜ 신항 P-OIL,부산 강서구 가락대로 1027,GS칼텍스,051-832-6717,Y,1825,1569,1429,-
8,부산광역시,구랑주유소,부산 강서구 가락대로 546 (구랑동),GS칼텍스,051-972-1571,Y,-,1569,1429,-
9,부산광역시,공항대로주유소,부산 강서구 공항로 1217 (대저1동),알뜰주유소,051-973-8122,Y,-,1569,1429,1250


In [35]:
result_df.to_excel('통합_주유소_데이터.xlsx', index=False)

In [36]:
gps_path = "./ref/부산_전체_주유소_위도경도.csv"
gps_data = pd.read_csv(gps_path, encoding="utf-8")
gps_data

,구,주소,상호,휘발유,경유,위도,경도
0,강서구,부산 강서구 낙동남로 432,해안주유소,1549.0,1399.0,35.111787,128.867467
1,강서구,부산 강서구 낙동남로 127 (송정동),신창에너지㈜직영 송정셀프주유소,1549.0,1409.0,35.101886,128.836964
2,강서구,부산 강서구 낙동북로 364,은마석유 경기장주유소,1557.0,1417.0,35.211911,128.968054
3,강서구,부산 강서구 낙동북로 209 (대저1동),동방석유㈜직영 대저주유소,1557.0,1417.0,35.214206,128.951693
4,강서구,부산 강서구 낙동북로 142,우리2주유소,1557.0,1417.0,35.215329,128.944364
...,...,...,...,...,...,...,...
342,사상구,부산 사상구 낙동대로 1320,대림주유소,1679.0,1549.0,35.177934,128.976559
343,사상구,부산 사상구 낙동대로 935,광신홀딩스㈜ 직영 낙동대로주유소,1689.0,1544.0,35.144005,128.969270
344,사상구,부산 사상구 학감대로 73 (학장동),행복한셀프주유소,1689.0,1549.0,35.138868,128.991863
345,사상구,부산 사상구 낙동대로 1380,지에스칼텍스(주)서부주유소,1739.0,1599.0,35.183325,128.976685


In [42]:
c_data = gps_data.sort_values('휘발유', ascending=True).head(5)
e_data = gps_data.sort_values('휘발유', ascending=False).head(5)

In [1]:
#!pip install folium

In [46]:
import folium

m = folium.Map(location=[35.1796, 129.0756], zoom_start=12)

for _, row in c_data.iterrows():
    popup_html = f"""
    <div style="font-size:13px;">
        <b>{row['상호']}</b><br>
        휘발유: {int(row['휘발유'])}원
    </div>
    """
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=folium.Popup(popup_html, max_width=200),
        icon=folium.Icon(color='blue')
    ).add_to(m)

for _, row in e_data.iterrows():
    popup_html = f"""
    <div style="font-size:13px;">
        <b>{row['상호']}</b><br>
        휘발유: {int(row['휘발유'])}원
    </div>
    """
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=folium.Popup(popup_html, max_width=200),
        icon=folium.Icon(color='red')
    ).add_to(m)

m
